In [60]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
import os 

from tqdm import tqdm

In [2]:
# pip install imdby

In [3]:
# pip install IMDbPY

In [4]:
# pip install cinemagoer

In [11]:
df = pd.read_csv('data/moscow_cinemas.csv')
df.head()

,Cinema,First day of screening,Screening Days,Title (original),IMDB ID
0,Метрополь / Metropol',01/01/1946,50.0,Разгром Японии,171687
1,Москва / Moskva,01/01/1946,15.0,Разгром Японии,171687
2,Родина / Rodina,01/01/1946,3.0,Разгром Японии,171687
3,Востоккино / Vostokkino,01/01/1946,45.0,Разгром Японии,171687
4,Баррикады / Barrikady,01/01/1946,2.0,Разгром Японии,171687


In [107]:
df['IMDB ID'].nunique()

829

In [16]:
df['IMDB ID'].unique()[:5]

array(['171687', '39925', 'Unknown', '33095', '37520'], dtype=object)

In [59]:
from collections import defaultdict 

movie_info = defaultdict(list)
all_info = {}

for imdb_id in tqdm(df['IMDB ID'].unique()): 
    try: 
        if 'unknown' in str(imdb_id).strip().lower(): 
            continue 

        ia = Cinemagoer()
        feature_movie = ia.get_movie(imdb_id) 

        movie_info['title'].append(df[df['IMDB ID'] == imdb_id]['Title (original)'].values[0])
        movie_info['genre'].append(feature_movie['genres'] if 'genres' in feature_movie else np.nan)

    #     print(feature_movie['country'])
        try: 
            movie_info['country'].append(feature_movie['country']) 
        except: 
            print('oops, country not stated') 
            movie_info['country'].append(np.nan)

        movie_info['rating'].append(feature_movie['rating'] if 'rating' in feature_movie else np.nan)
        movie_info['director'].append(feature_movie['director'] if 'director' in feature_movie else np.nan)

        all_info[imdb_id] = feature_movie
    except: 
        print('something went wrong') 
        continue 

  2%|▋                                         | 13/829 [00:38<43:09,  3.17s/it]

oops, country not stated


  4%|█▌                                        | 32/829 [01:42<43:40,  3.29s/it]2023-04-19 14:36:17,694 CRITICAL [imdbpy] /home/timcha/.local/lib/python3.10/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0029405/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': URLError(ConnectionResetError(104, 'Connection reset by peer'))},); kwds: {}
Traceback (most recent call last):
  File "/usr/lib/python3.10/urllib/request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/usr/lib/python3.10/http/client.py", line 1282, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.10/http/client.py", line 1328, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.10/http/client.py", line 1277, in endheaders
    self._send_outpu

something went wrong


100%|█████████████████████████████████████████| 829/829 [44:15<00:00,  3.20s/it]


In [62]:
with open('data/movies_all_info.pickle', 'bw') as f: 
    pickle.dump(all_info, f)

In [67]:
movie_info.keys()

dict_keys(['title', 'genre', 'country', 'rating', 'director'])

In [111]:
res['year'] = [all_info[key]['year'] for key in all_info]

In [95]:
res = pd.DataFrame(movie_info)

In [96]:
res.dtypes

title        object
genre        object
country      object
rating      float64
director     object
dtype: object

In [86]:
all_genres = [] 
for genre in res['genre'].values:
    if genre is not np.nan:
        all_genres.extend(genre)

In [87]:
len(set(all_genres))

23

In [75]:
for key in movie_info: 
    print(f'key: {key}, len: {len(movie_info[key])}')

key: title, len: 817
key: genre, len: 817
key: country, len: 817
key: rating, len: 817
key: director, len: 817


In [98]:
for i in res['director'][0]: 
    print(i['name'])

Iosif Kheifits
Aleksandr Zarkhi


In [101]:
res['director'] = res['director'].apply(lambda x: ', '.join([i['name'] for i in x]))

In [104]:
res['genre'] = res['genre'].apply(lambda x: ', '.join([i for i in x]) if x is not np.nan else np.nan) 

In [105]:
res['country'] = res['country'].apply(lambda x: ', '.join([i for i in x]) if x is not np.nan else np.nan)

In [112]:
res.to_excel('final_version.xlsx')

In [90]:
genre2idx = {genre: i for i, genre in enumerate(list(set(all_genres)))}

In [93]:
ohe_matrix = np.zeros((res.shape[0], len(set(all_genres))))
ohe_matrix.shape

(817, 23)

In [ ]:
for i, row in df.iterrows(): 
    i